In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download pt_core_news_sm

In [16]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

In [11]:
# carregando modelos salvos
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/modelo_skipgram.txt")

In [19]:
artigo_treino = pd.read_csv('/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/treino.csv')
artigo_teste = pd.read_csv('/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/teste.csv')

# Pré Processamento

In [7]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
  doc = nlp(texto)
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())
  return tokens_validos

texto = "Rio de Janeiro 123 ** @ cidade maravilhosa !"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [15]:
def combinacao_de_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)

  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn)

    except KeyError:
      pass


  return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
vetor_texto.shape, vetor_texto

((300,),
 array([-0.11607877, -0.34202099,  0.49601896, -1.20712078,  0.03994992,
         1.94143014, -1.89674586,  0.33286209,  0.64318092, -0.51436861,
         0.41572537,  0.7094056 , -1.26321053,  0.18414581, -0.9783653 ,
        -2.27229333, -0.40744796, -1.86127752, -0.13888195,  0.06942251,
        -0.22775149, -1.16429922,  0.67629124,  0.08126878,  0.63199276,
        -1.97258108,  0.00536719, -0.6487813 , -0.47326058,  1.40060127,
         0.41343138, -3.10279858, -0.52750862,  1.34813474,  1.54102479,
        -0.65355068, -2.70351598, -1.41391912,  0.27802617, -0.02885799,
        -1.02226484,  1.17564747,  0.94138348,  0.20651188, -1.80907165,
        -0.18533987,  3.18678612, -0.89231944,  1.41172369, -0.9026517 ,
        -0.98379958, -2.67319816,  0.2834765 , -2.61265382,  0.87042455,
         1.47705839,  0.56058644, -0.06154514,  1.13848461,  0.5407267 ,
        -0.90345231, -1.34692733,  0.84281583,  0.55721696,  1.14206697,
         0.2467421 ,  0.82463691,  0.69698

In [29]:
def matriz_vetores(textos, modelo): #demorou 20min
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

#cbow
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

# sg
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
#matriz_vetores_treino_cbow, matriz_vetores_teste_cbow

In [22]:
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


# Classificador
Decidir qual modelo é melhor cbow ou skipgram

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)

  return RL


In [28]:
RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.12      0.82      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.78      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.78      0.80     20513



In [30]:
RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.72      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.15      0.89      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.76      0.85      0.80      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



# Salvando Modelo de Classificação

In [31]:
import pickle

with open('/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/rl_cbow.pkl', 'wb') as file:
  pickle.dump(RL_cbow, file)


In [32]:
with open('/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/rl_skipgram.pkl', 'wb') as file:
  pickle.dump(RL_sg, file)